# TP1 HomeWork

In [2]:
import numpy as np
from main import *

tps écoulé (gradient_rho_constant): 0.015623092651367188
[-0.69056385  0.15469913 -0.61430107  0.4909626  -0.04630803]


 **We define an adaptative version of the gradient descend algorithm** 

In [3]:
def gradient_rho_adaptatif(fun, fun_der, U0, rho, tol,args):
    
    # Fonction permettant de minimiser la fonction f(U) par rapport au vecteur U 
    # Méthode : gradient à pas fixe
    # INPUTS :
    # - han_f   : handle vers la fonction à minimiser
    # - han_df  : handle vers le gradient de la fonction à minimiser
    # - U0      : vecteur initial 
    # - rho     : paramètre gérant l'amplitude des déplacement 
    # - tol     : tolérance pour définir le critère d'arrêt
    # OUTPUT : 
    # - GradResults : structure décrivant la solution


    itermax=10000  # nombre maximal d'itérations 
    xn=U0
    f=fun(xn,*args) # point initial de l'algorithme
    it=0         # compteur pour les itérations
    converged = False;
    
    while (~converged & (it < itermax)):
        it=it+1
        dfx=fun_der(xn,*args)# valeur courante de la fonction à minimiser
        
        xnp1=xn-rho*dfx
        fnp1 = fun(xnp1,*args)
        
        if fnp1 < f :
            
            if abs(fnp1-f)<tol:
                converged = True
                
            xn, f = xnp1, fnp1
            rho *= 2
        else :
            rho /= 2

    GradResults = {
            'initial_x':U0,
            'minimum':xnp1,
            'f_minimum':fnp1,
            'iterations':it,
            'converged':converged
            }
    return GradResults

In [4]:
gradient_rho_adaptatif(f1,df1,x0,rho=1,tol=1e-6,args=(B,S))

{'initial_x': array([1., 1., 1., 1., 1.]),
 'minimum': array([-0.67000244,  0.1425606 , -0.61584071,  0.47887395, -0.02041732]),
 'f_minimum': -1.8361443391078156,
 'iterations': 386,
 'converged': True}

## On va plot le nombre d'itération par rapport à rho

## Méthodes Quasi-Newton

In [5]:
from scipy.optimize import minimize

In [6]:
minimize(fun = f1, x0 = x0, args=(B,S), method='BFGS', tol=1e-6)

      fun: -1.836962311965236
 hess_inv: array([[ 0.50041674, -0.21238233,  0.05674326, -0.28046232,  0.50962222],
       [-0.21238233,  0.22402876,  0.03714748,  0.11546007, -0.34089541],
       [ 0.05674326,  0.03714748,  0.12473992, -0.04895957, -0.09129864],
       [-0.28046232,  0.11546007, -0.04895957,  0.20494128, -0.28975672],
       [ 0.50962222, -0.34089541, -0.09129864, -0.28975672,  0.77797771]])
      jac: array([ 0.00000000e+00,  8.94069672e-08, -2.98023224e-08,  1.49011612e-08,
        1.49011612e-08])
  message: 'Optimization terminated successfully.'
     nfev: 91
      nit: 10
     njev: 13
   status: 0
  success: True
        x: array([-0.6960314 ,  0.15793136, -0.61407083,  0.49414156, -0.05345805])

## Résolution analytique

In [7]:
np.linalg.eigvals(S)

array([45.48200751, 14.49627115,  0.34399006,  1.89569112,  7.21954015])

solution analytique existe

In [8]:
S_1 = np.linalg.inv(S)
solution = 0.5*S_1*B
np.abs(f1(solution, B, S) - minimize(fun = f1, x0 = x0, args=(B,S), method='BFGS', tol=1e-6)['fun'])

1.6653345369377348e-14

## Optimisation sous contraintes

In [9]:
minimize(fun = f1, x0 = x0, args=(B,S), method='SLSQP', bounds=[(0,1)]*5, tol=1e-6)

     fun: -0.13853161426486305
     jac: array([5.65140147e-01, 1.70477480e-03, 4.86906105e+00, 1.77648850e-03,
       2.70384207e-01])
 message: 'Optimization terminated successfully.'
    nfev: 65
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([0.00000000e+00, 1.26984956e-01, 0.00000000e+00, 1.94536138e-02,
       2.16628331e-15])

In [10]:
def f2(U,S):
    n=U.shape[0]
    U=np.matrix(U)
    U.shape=(n,1)
    fU = np.transpose(U) * S * U + np.transpose(U) * np.exp(U);
    return float(fU)

In [11]:
minimize(fun = f2, x0 = x0, args=(S), method='SLSQP', bounds=[(0,1)]*5, tol=1e-6)

     fun: 2.976067656614975e-15
     jac: array([1.00000018, 1.00000013, 1.00000035, 1.00000023, 1.00000022])
 message: 'Optimization terminated successfully.'
    nfev: 28
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([1.72929872e-16, 2.17964615e-15, 5.95736056e-16, 0.00000000e+00,
       2.77555756e-17])

## Optimisation sous contraintes et pénalisation

In [12]:
#fonction de pénalisation classique 
Beta = lambda u : np.sum(np.maximum(u-1, 0)**2 + (np.maximum(-u,0))**2)
print(Beta(x0), Beta(x0 - 2))

0.0 5.0


In [13]:
epsilon, start = 1/2, x0
for k in range(100):
    f1_penal = lambda U : f1(U,B,S) + (1/epsilon)*Beta(U)
    start = minimize(fun = f1_penal, x0 = start, method='BFGS', tol=1e-6)['x']
    epsilon /= 2
    
print(f1(start, B , S))
print(start)

-0.13853173459306484
[2.27345020e-09 1.26885930e-01 2.95427674e-10 1.94103684e-02
 9.25641783e-10]


In [14]:
epsilon, start = 1/2, x0
for k in range(1000):
    f2_penal = lambda U : f2(U,S) + (1/epsilon)*Beta(U)
    start = minimize(fun = f2_penal, x0 = start, method='BFGS', tol=1e-6)['x']
    epsilon /= 2
    
print(f2(start , S))
print(start)

8.152761402346261e-11
[7.19742181e-17 1.09058344e-11 1.98749448e-11 2.79335748e-11
 2.28131881e-11]


## Méthodes duales 